# Chapter 8 - Generalization and Function Approximation

This chapter explores methods for Reinforcement Learning beyond the simple, tabular environments presented so far, in situations where the state space, the action space or both are larger (or infinite) and cannot be represented effectively by the tabular framework. Generalization is achieved via function approximation algorithms from Supervised Learning which are adapted for learning state- or action-value functions. Gradient-descent versions of on-line TD($\lambda$), Sarsa($\lambda$) and Watkin's Q($\lambda$) algorithms are given, while off-policy methods that also bootstrap are avoided since they didn't have good convergence properties (see last exercise).

This chapter is much more developed in the Second Edition of the book (Chapters 9-13). It is strongly recommended to read those instead, for more up-to-date ideas, algorithms and theoretical properties.

### Exercise 8.1

Show that table-lookup TD($\lambda$) is a special case of general TD($\lambda$) as given by equations (8.5 - 8.7).

__Answer__: Since we are in the tabular case, there is a finite number of states, $\lvert \mathcal{S} \rvert$. So we can index the states arbitrarily by an integer like $s_1$, $s_2$, ..., $s_{\lvert \mathcal{S} \rvert}$ (there is no temporal meaning associated with this index, it's just a way to uniquely identify the states of the MDP). We can now represent the table-lookup values by using a simple vector $\theta \in \mathbb{R}^{\lvert \mathcal{S} \rvert}$ and define the value of any state $s_k \in \mathcal{S}$ as $V(s_k) \doteq \theta(k)$, the $k$-th component of the vector $\theta$. If we refer to the value of a state at a certain time step $t$, we can also write $V_t(s_k) \doteq \theta_t(k)$, to indicate that the vector $\theta$ itself is updated.

The $\delta_t$ remains the same as in equation (8.6) and the eligibility trace $\vec{e}$ is now a vector of dimension ${\lvert \mathcal{S} \rvert}$, containing the traces for all states $s \in \mathcal{S}$ at once. From the definition of $V$, the gradient of $V_t(s_k)$ w.r.t $\theta$ is a one-hot vector of zeros on all positions except in position $k$ where it is $1$.

Therefore, the update of $V$ after each step is replaced with identical results by simply updating the vector $\theta$ according to equation (8.5), for all steps at once. So the general TD($\lambda$) can easily represent a tabular TD($\lambda$) with these simple conventions.

### Exercise 8.2

_State aggregation_ is a simple form of generalizing function approximation in which states are grouped together, with one table entry (value estimate) used for each group. Whenever a state in a group is encountered, the group's entry is used to determine the state's value, and when the state is updated, the group's entry is updated. Show that this kind of aggregation is a special kind of gradient method such as (8.3).

__Answer__: Let's denote as usually the set of states $\mathcal{S}$ (not necessarily finite), the set of groups $\mathcal{G}$ (must be finite) and a function $g : \mathcal{S} \to \mathcal{G}$, that maps each state $s$ to the group it belongs.

Now let's introduce a vector $\vec{\theta_t} \in \mathbb{R}^{\lvert \mathcal{G} \rvert}$, which represents the set of all group values at any time step $t$. In other words, $\vec{\theta_t} = \Big( \theta_t(1), \theta_t(2), ..., \theta_t(\lvert \mathcal{G} \rvert) \Big) \doteq \Big( V_t(grp_1), V_t(grp_2), ..., V_t(grp_{\lvert \mathcal{G} \rvert}) \Big)$, where we employed the usual value function $V$ applied to a group, not to a state. But since the state aggregation method implies that the value of a state is always linked to the value of its group, by a slight abuse of notation, we can also consider true that $V(s) = V(g(s))$ for all $s$.

With this notation in place, we can now write the update rule for this method, which applies to groups, not states. Thus at any time step $t$, for any state $s$, its group's value $V(g(s))$ is updated as:

$$
\begin{aligned}
    V_{t+1}(g(s)) & \leftarrow \begin{cases} 
        V_{t}(g(s)) + \alpha \Big[ R_t - V_t(g(s)) \Big] && \text {if } s = s_t \\
        V_{t}(g(s)) && \text {if } s \ne s_t \\
    \end{cases} \\ \\
    & = V_{t}(g(s)) + \alpha \Big[ R_t - V_t(g(s_t)) \Big] \cdot \mathcal{I}_{ss_t} \text {, where } \mathcal{I}_{ss_t} \text { is the usual indicator}. \\
    \\
    \text {Which by definition is equivalent to:} \\
    \theta_{t+1}(g(s)) & = \theta_{t}(g(s)) + \alpha \Big[ R_t - V_t(g(s_t)) \Big] \cdot \mathcal{I}_{ss_t} \text {, for all } s \\
\end{aligned}
$$

Since we defined the vector $\vec{\theta_t}$ to contain all group values $V_t(\cdot)$, we can now write a vectorized version of this update, such that the values of all groups are updated at once. Let's define $\vec{\mathcal{I}}_{gs_t} = \Big( 0, 0, ..., 1, ..., 0 \Big)$, a one-hot vector of size $\lvert \mathcal{G} \rvert$, that indicates to which group belongs $s_t$. The vectorized update rule then becomes:

$$
\begin{aligned}
    \vec{\theta}_{t+1} & = \vec{\theta_{t}} + \alpha \Big[ R_t - V_t(g(s_t)) \Big] \cdot \vec{\mathcal{I}}_{gs_t} \\
    & = \vec{\theta_{t}} + \alpha \Big[ R_t - V_t(g(s_t)) \Big] \cdot \nabla_{\vec{\theta_t}} V_t(g(s_t)) \qquad (1) \\
    & = \vec{\theta_{t}} + \alpha \Big[ R_t - V_t(s_t) \Big] \cdot \nabla_{\vec{\theta_t}} V_t(s_t) \text {, which is exactly (8.3)}. \\
\end{aligned}
$$

Notice the equality in (1) is true because by definition $V_t(g(s_t))$ depends only on a single component of the vector $\vec{\theta}$, namely $\theta(g(s_t))$, so its gradient w.r.t $\vec{\theta_t}$ is simply the one-hot vector: $\Big( \dfrac {\partial \theta(g(s_t))} {\partial \theta(1)}, \dfrac {\partial \theta(g(s_t))} {\partial \theta(2)}, ..., \dfrac {\partial \theta(g(s_t))} {\partial \theta(g(s_t))}, ..., \dfrac {\partial \theta(g(s_t))} {\partial \theta(\lvert \mathcal{G} \rvert)} \Big) = \Big( 0, 0, ..., 1, ..., 0 \Big) = \vec{\mathcal{I}}_{gs_t}$.

Also, for the last equality we use the fact that $V_t(s_t) \doteq V_t(g(s_t))$. This completes the proof that State aggregation can be regarded as a gradient method.

### Exercise 8.3

The equations given in this section are for the on-line version of the gradient-descent TD($\lambda$). What are the equations for the off-line version? Give a complete description specifying the new approximate value function at the end of an episode, $V'$, in terms of approximate value function used during the episode, $V$. Start by modifying a forward view equation for TD($\lambda$), such as (8.4).

__Answer__: In the off-line version of gradient-descent TD($\lambda$), the updates to the vector $\vec{\theta}$ are accumulated on the side and applied at the end of the episode. Also, the values of $\vec{\theta}$ remain constant throughout the episode and implicitly the values of $V$ (which for clarity we'll write as a function $f$ of both $\theta$ and $s$), i.e. $V(s) = f(\theta, s) \text { and } V'(s) = f(\theta', s)$. With this notation then equation (8.4) becomes (aggregated over the entire episode):

$$
\begin{aligned}
    & \vec{\theta'} = \vec{\theta} + \sum_{t=0}^{T-1} \alpha \left[ R_t^{\lambda} - f(\theta, s_t) \right] \nabla_{\vec{\theta}} f(\theta, s_t) \\
    \implies V'(s) & = f(\vec{\theta'}, s) \\ 
    & = f \Bigg( \vec{\theta} + \sum_{t=0}^{T-1} \alpha \left[ R_t^{\lambda} - f(\theta, s_t) \right] \nabla_{\vec{\theta}} f(\theta, s_t), s \Bigg) \\
    & = f \Bigg( \vec{\theta} + \sum_{t=0}^{T-1} \alpha \left[ R_t^{\lambda} - V(s_t) \right] \nabla_{\vec{\theta}} V(s_t), s \Bigg) \\
\end{aligned}
$$

### Exercise 8.4

For off-line updating, show that equations (8.5 - 8.7) produce updates identical to (8.4).

__Answer__: The equations (8.5 - 8.7) describe the backward view of gradient-descent TD($\lambda$), while (8.4) stands for the forward view of the same algorithm. The proof follows the same line as the one in Section 7.4. We need to show that the vector $\vec\theta$ is updated at the end of the episode by the same amount, i.e. the vectors $\sum_{t=0}^{T-1} \vec{\Delta\theta}_t^{TD}$ from the backward view and $\sum_{t=0}^{T-1} \vec{\Delta\theta}_t^{\lambda}$ from the forward view are equal. Since the update is done off-line, it means that the vector $\vec\theta$ and implicitly the state values $V$ remain unchanged for the duration of the episode. So we can safely drop the subscript $t$ to simplify the notation. Similarly for $\nabla_{\vec\theta} V$, the gradient of $V$ w.r.t. $\vec\theta$, we can safely drop the $\vec\theta$ subscript, as it is implied. 

Now let's work on the left term, by first writing the non-recursive value of the eligibility traces vector $\vec e$:

$$
\begin{aligned}
    \vec{e}_{t} & = \gamma \lambda \vec{e}_{t-1} + \vec{\nabla V}(s_t) \\
    & = \gamma \lambda \left( \gamma \lambda \vec{e}_{t-2} + \vec{\nabla V}(s_{t-1}) \right) + \vec{\nabla V}(s_t) \\
    & = ... \\
    & = \sum_{k=0}^{t} (\gamma \lambda)^{t-k} \vec{\nabla V}(s_k) \\
\end{aligned}
$$

Thus the left term becomes, from (8.5):

$$
\begin{aligned}
    \sum_{t=0}^{T-1} \vec{\Delta\theta}_t^{TD} & = \sum_{t=0}^{T-1} \alpha \delta_t \vec{e}_{t} \\ \\
    & = \sum_{t=0}^{T-1} \alpha \delta_t \sum_{k=0}^{t} (\gamma \lambda)^{t-k} \vec{\nabla V}(s_k) \\ \\
    & = \sum_{k=0}^{T-1} \alpha \sum_{t=0}^{k} (\gamma \lambda)^{k-t} \delta_t \vec{\nabla V}(s_t) \\ \\
    & = \sum_{t=0}^{T-1} \alpha \sum_{k=t}^{T-1} (\gamma \lambda)^{k-t} \delta_k \vec{\nabla V}(s_t) \\ \\
    & = \sum_{t=0}^{T-1} \alpha \vec{\nabla V}(s_t) \sum_{k=t}^{T-1} (\gamma \lambda)^{k-t} \delta_k \qquad (1) \\
\end{aligned}
$$

Turning now to the right term, as per (8.4), every increment of $\theta$ contains the scalar factor $\left[ R_t^{\lambda} - V(s_t) \right]$ for all steps $t$. I won't repeat the whole reasoning here, as it is identical to the one in Section 7.4, but this factor is proven to be exactly equal (for off-line updating) to $\sum_{k=t}^{T-1} (\gamma \lambda)^{k-t} \delta_k$. We can now write:

$$
\begin{aligned}
    \sum_{t=0}^{T-1} \vec{\Delta\theta}_t^{\lambda} & = \sum_{t=0}^{T-1} \alpha \left[ R_t^{\lambda} - V(s_t) \right] \vec{\nabla V}(s_t) \\ \\
    & = \sum_{t=0}^{T-1} \alpha \vec{\nabla V}(s_t) \sum_{k=t}^{T-1} (\gamma \lambda)^{k-t} \delta_k \qquad (2) \\
\end{aligned}
$$

We notice that (1) and (2) are identical, which completes the proof that forward and backward views of off-line TD($\lambda$) are equivalent also for the gradient-descent algorithm.

### Exercise 8.5

How could we reproduce the tabular case within the linear framework?

__Answer__: In the tabular case we have with a finite set of states $s_i, \text { for } i \in \{1, 2, ..., \lvert \cal{S} \rvert \}$. Let's consider the feature vector $\vec\phi_{s_i} = (0, 0, ..., 1, ... 0)^T \in \mathbb{R}^{\lvert \cal{S} \rvert}$, a one-hot vector such that $\phi_{s_i}(i) = 1 \text { for all } i \text{ and } \phi_{s_i}(j) = 0 \text{ for all } i \ne j$.

With the feature vector constructed in this way, then $V_t(s_i) = {\vec\theta_t}^T \vec\phi_{s_i} = \theta_t(i) \phi_{s_i}(i) = \theta_t(i)$. So the state-value for the $i$-th state at time $t$ is the same as the $i$-th component of vector $\vec\theta_t$. At the same time, the gradient $\nabla_{\vec\theta_t} V_t(s_i) = \vec\phi_{s_i}$.

So the usual TD update rule for the tabular case can be equivalently written for any state $s_i$ at time $t$:

$$
\begin{aligned}
    & V_{t+1}(s_i) = V_{t}(s_i) + \alpha [R_t - V_{t}(s_i)] \cal{I}_{s_is_t} \\
    \iff & \theta_{t+1}(i) = \theta_{t}(i) + \alpha [R_t - \theta_{t}(i)] \phi_{s_t}(i) \\
    & \text {which we can now vectorize for all } i \\
    \iff & \vec\theta_{t+1} = \vec\theta_{t} + \alpha [R_t - \vec\theta_{t}] \vec\phi_{s_t} \\
    & \qquad = \vec\theta_{t} + \alpha [R_t - \vec\theta_{t}] \nabla_{\vec\theta_t} V_t(s_t) \\
\end{aligned}
$$

Which is exactly the equation (8.3) for the general gradient-descent methods.

### Exercise 8.6

How could we reproduce the state aggregation case (see Exercise 8.2) within the linear framework?

__Answer__: Very similar to the above for the tabular case, only this time we replace states by their corresponding group. Let the groups be $g_1, g_2, ..., g_{\lvert \cal{G} \rvert}$ and $\vec\phi_{g_i} \in \mathbb{R}^{\lvert \cal{G} \rvert}$ similarly a one-hot vector such that $\phi_{g_i}(i) = 1 \text{ for all } i \text{ and } \phi_{g_i}(j) = 0 \text{ for all } i \ne j$.

Similar to above, $V_t(g_i) = {\vec\theta_t}^T \vec\phi_{s_i} = \theta_t(i)$, and everything from the tabular case applies here too, but at the group level. So we end up with showing that the update rule for the tabular case is equivalent to the gradient-descent one (still at group level, as if groups were states).

### Exercise 8.7

Suppose we believe that one of two state dimensions is more likely to have an effect on the value function than is the other, that generalization should be primarily across this dimension rather than along it. What kind of tilings could be used to take advantage of this prior knowledge?

__Answer__: The tilings suitable to represent this knowledge would be formed of elongated tiles along the second dimension, and narrow along the primary dimension, such as to allow finer approximation of the learned function.

### Exercise 8.8

Describe how the actor-critic control method can be combined with gradient-descent function approximation.

__Answer__: The critic could learn a value function approximation of $V^{\pi}$ based on some parameter $\vec\theta^{critic}$, using the exact same method as described by equations (8.5 - 8.7), i.e. gradient-descent TD($\lambda$).

As for the actor, it learns a preference function, $p(s, a)$, which could also be a smooth approximation parameterized by a vector $\vec\theta^{actor}$. The update equations for it would be (backward view):

$$
\vec\theta^{actor}_{t+1} \leftarrow \vec\theta^{actor}_t + \beta \delta_t \vec{e}_t \\
\vec{e}_t \leftarrow \gamma \lambda \vec{e}_{t-1} + \nabla_{\vec\theta_t^{actor}} p(s_t, a_t)
$$

Notice that there are two different eligibility trace vectors, one for the actor and one for the critic.



### Exercise 8.9

Look up the paper by Baird (1995) and obtain his counterexample for Q-learning. Implement it and demonstrate the divergence.

Couldn't get the actual paper (behind paywall). The divergence due to bootstrapping in conjunction with function approximation and off-policy algorithms is much more developed in the 2nd edition of the book (see Section 11.3 The Deadly Triad).

# Second Edition

Whereas the problem of Generalization and Function Approximation covered a single chapter in the 1st edition, there are now 5 chapters (9-13) that treat different aspects of it in the 2nd edition, over more than 140 pages.

### Chapter 9 (2nd edition)

It covers the value-function evaluation via approximations and starts with defining the prediction objective, i.e. to minimize the MSE between the approximated and the true state values (weighted by a probability distribution over states that indicates how much we care about the error in each state), just as in the 1st edition, however a clear distinction is drawn between pure-gradient (SGD) methods and semi-gradient methdods, which are applied when bootstrapping. To clarify, in the notation from first edition, usually we update $\vec\theta$ by:

$$
    \vec\theta_{t+1} \leftarrow \vec\theta_{t} + \alpha [v_t - V_t(s_t)] \nabla_{\vec\theta_t}V_t(s_t)
$$

Where $v_t$ is any generic target, i.e. an unbiased estimate of the true $V^{\pi}(s_t)$. This works well when we use Monte Carlo return $R_t$ (which is an unbiased estimate), but not when we're bootstrapping, e.g. by using a $R_t^{\lambda}$ return or any n-step TD return, because they are some function of $\sum_{k=0}^{n} \gamma^{k} r_{t+k+1} + \gamma^{n+1} V_{\vec\theta_t} (s_{t+n+1}), n \ge 0$. So the target value itself depends of $\vec\theta$, therefore all bootstrapping methods like this are not true gradient methods, hence the name semi-gradient. There are also some detailed convergence proofs for true-gradient methods, but also for special cases of semi-gradient ones, e.g. when the function approximator is linear it converges to TD fixed points.

The next sections deal with feature construction techniques for the linear approximator functions, from polynomials to Fourier basis, Radial basis function (RBF), to coarse coding and tiling.

Section 9.7 is dedicated to neural networks, just introducing the generic concepts and ideas, nothing much specific to RL.

Section 9.9 is dedicated to so called memory-based function approximation, which are non-parametric methods relying on storing some past experiences (states, etc.) and update the value for a current _query_ state $s$ based on other similar states in memory. Examples are k-nearest neighbours and k-d trees.

Section 9.10 briefly describes kernel methods, as another way to represent state values and relations between them, e.g. how relevant is knowledge about some state $s'$ for the query state $s$. Can be combined with other feature construction techniques and memory-based methods.

Section 9.11 introduces the idea that some states (and/or their values) might be of greater / lesser importance than others to the ultimate goal of RL, which is to find an optimum policy. Function approximation methods could use this by weighting each update in relation to some newly introduced variables - _interest_ (in a state or state-action pair) and _emphasis_.

### Chapter 10 (2nd edition)

This chapter deals with the Control problem for on-policy approximation methods, which includes parametric action-value functions, policy improvement (following GPI) and action selection (e.g. $\epsilon$-greedy). All these are first dealt with for episodic tasks, then extended to the continuing case by switching to a new RL setting - the average-reward formulation and differential values. The undiscounted continuing tasks had been also briefly treated in the 1st edition in Chapter 6, Section 6.7, but only for the tabular setting and under slightly different names - differential values were called _relative values_ and the given algorithm was an off-policy TD algorithm called _R-learning_, while Exercise 6.11 (1st edition) asked for an on-policy algorithm.

In Section 10.4 (2nd edition), there is a great explanation why discounted returns have no influence over the selection of policies in continuing tasks, i.e. "The futility of discounting in continuing problems".

### Chapter 11 (2nd edition)

This chapter explores off-policy methods with function approximation. It starts by explaining why off-policy methods become significantly more difficult when function approximations are used in contrast with the tabular setting, namely due to convergence issues. In the first part of the chapter the same idea of importance sampling, first introduced in Section 5.5 (1st edition) is employed with function approximation in an identical form.

For the second part, the instability of combining function approximation with bootstrapping was also touched on in Sections 8.5 - 8.6 in the 1st edition, but here the problem is treated in much more detail and it also discusses possible solutions.

The first approach is to apply true-gradient-descent methods to the Bellman error, via different approximations. First, there's the __naive residual-gradient algorithm__, which simply tries to minimize the TD error in a true-gradient manner. However there are simple counterexamples where it fails to converge to the correct state values. Then trying to minimize the Bellman error directly, i.e. the expectation of the TD error, via the __residual-gradient algorithm__, which is also a true-gradient method. It fails in a similar way as the first one. Section 11.6 then explains why the Bellman error is fundamentally __not learnable__: different MDPs can generate the exact same sequence (or distribution) of data. This directs the attention towards the _Projected Bellman error_, which is learnable via SGD. Thus two related _Gradient-TD_ algorithms are presented: __GTD2__ and the slightly better one __TD(0) with gradient correction (TDC) or GTD(0)__.

In Section 11.8 a second approach for off-policy learning is presented, __Emphatic-TD__ methods. The main idea is to re-weigh the states distribution due to the behaviour policy to match that of the target policy, in the same manner as importance sampling ratios are used to adjust state transition probabilities. However, this method suffers from high variance and doesn't seem to be of practical value.

### Chapter 12 (2nd edition)

This chapter regards Eligibility Traces and we've already covered it at the end of Chapter_7 notebook.

### Chapter 13 (2nd edition)

This chapter is entirely new in the 2nd edition, treating the topic of learning by __Policy Gradients__.

Until now we've dealt with estimating state- or action-value functions, from which a (optimal) policy could then be derived for control. A different approach with interesting ramifications is to directly approximate the policy itself (and optimize it via gradient ascent). This is what Policy Gradients methods do - instead of using function approximators to parameterize the state- or action-value function with a vector $\vec\theta$, they parameterize the policy, $\pi(a | s, \theta) = Pr \left\{ a_t = a | s_t = s, \theta_t = \theta \right\}$, as a smooth function differentiable w.r.t. $\theta$.
For every policy $\pi(\cdot | \cdot, \theta)$, a scalar function $J(\theta)$ is introduced that evaluates the goodness of that policy in a given environment. And at every step, the policy parameter vector $\theta$ is updated as to increase the performance $J(\theta)$ in the direction of its (stochastic) gradient:

$$
    \theta_{t+1} \leftarrow \theta_t + \alpha \widehat{\nabla_{\theta_t} J_{\theta_t}}
$$

There are a number of advantages of this method, like the convenience of obtaining an exact policy that can be directly used for control, the ease of using it for continuous or large action spaces, or learning an optimal stochastic policy which can't be otherwise expressed via a deterministic or $\epsilon$-soft policy.

The apparent challenge with policy approximation is that by changing its parameters we are changing both _action selection_ and _state distribution_ at the same time. The former is straight-forward to address, but the latter is also a function of the environment and usually considered unknown. However, this difficulty is overcome via the __Policy Gradient Theorem__ (proofs in Section 13.2 for the episodic tasks and in Section 13.6 for the continuing case).

The remaining section of Chapter 13 present a series of algorithms starting with (Monte Carlo) REINFORCE, which is the base algorithm for episodic policy gradient learning, then gradually introducing REINFORCE with a Baseline (to reduce variance), followed by one-step Actor-Critic (which gets rid of the need for Monte Carlo and uses bootstrapping similar to TD methods), further generalized to Actor-Critic with Eligibility traces, for both episodic and continuing tasks.

The last Section 13.7 briefly discusses ways to deal with infinite or large action spaces, by learning a (parameterized) probability density function over the action space instead of a probability over discrete actions. A practical way to represent the density function is to start from a normal distribution and parameterize its mean and standard deviation (as separate functions).